In [ ]:
# feature selection using BorutaShap
from sklearn.base import clone

model_BS = clone(model)

Feature_Selector = BorutaShap(model=model_BS, 
                              importance_measure='shap', 
                              classification=True, 
                              percentile=100, 
                              pvalue=0.05)

Feature_Selector.fit(X=X_train_features, 
                     y=y_train, 
                     n_trials=100, 
                     sample=False, 
                     train_or_test = 'train', 
                     normalize=True, 
                     verbose=False, 
                     random_state=SEED)

In [ ]:
# boruta plot

Feature_Selector.plot(X_size=15,
                      figsize=(15,45),
                      which_features='all')

In [ ]:
# dimensionality reduction using feature importance

df_imp = pd.DataFrame({'imp':model.feature_importances_}, index = model.get_booster().feature_names)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()

feat_num = df_imp.shape[0]
print("total number of features =", feat_num)
df_imp

In [ ]:
auc_list = [mean(scores)]
for i in range(0,feat_num+1):
  features = df_imp.index[i:(feat_num+1)].to_list()
  mark_feature = X_train_features.columns[X_train_features.columns.isin(features)]
  X_train_new = X_train_features[mark_feature].copy()
  model_adj = XGBClassifier(colsample_bytree=0.6, gamma=0.8, learning_rate=0.01, max_depth=3, 
                          min_child_weight=6, n_estimators=500, objective="binary:logistic",
                          reg_alpha=0.5, reg_lambda=2, scale_pos_weight=14, subsample=0.2, 
                          nthread=6, random_state=SEED)
  scores_adj = evaluate_model(X_train_new, y_train, model_adj)
  print('%d, AUC = %.3f (%.3f)' % (i, mean(scores_adj), std(scores_adj)))
  auc_list.append(mean(scores_adj))
  #if (max(auc_list)-mean(scores_adj)) >= 0.005:
  #  break
  #else:
  #  auc_list.append(mean(scores_adj))

# plot
plt.plot(auc_list)